---
---

<center><h1> Setting up the Environment.</h1></center>

---


#### `Generate Refund Data`

---

In this notebook, we will generate random refund request data for the `refund_detail` table.

---

In the `refund_detail` table, we have the following columns:

- **`ticket_id`**    : Randomly generated string for each of the refund request. <br>
- **`user_name`** : Random user name , we will assume that it is entered by the user on our refund request page. <br>
- **`transaction_id`**  : Randomly selected transaction id from the transaction table. <br>
- **`transaction_amount`**     : Transaction amount, it will also entered by the user. <br> 
- **`ticket_raise_time`**   : Current time will be used as the ticket raise time. <br>

---



---

Every 10-20 seconds a row in added into the refund request table. This is completely customizable. We have selected this time period so that we will be able to test our pipleine quickly.

----

#### `Importing the Required Libraries`

---

In [5]:
# importing the required libraries
import time
import random
import uuid
import pandas as pd
import mysql.connector as mysql

---

#### `Define the function to generate refund requests.`

---

In [6]:
# define the generate_refund function
def generate_refund(transactions):
    # randomly generate the ticket id using uuid
    ticket_id = "TICKET-" + str(uuid.uuid4())
    
    # select any random transaction id from the transaction id list
    random_index = random.randint(0,len(transactions)-1)
    transaction_id = transactions[random_index][0]
    
    # create a random user name of 5 characters
    user_name  = str(''.join(random.choices("ABC",k=1))) + str(''.join(random.choices("ibruegbieiurgeriuger", k = 4)))
    
    # create a random transaction amount
    transaction_amount = transactions[random_index][1]
    refund_status = "NEW"
    
    # current time as the ticket raise time
    ticket_raise_time = str(pd.datetime.now())
    
    # we will add error in 20 percent of the refund request.
    # if the random error is 1, then it is a valid request
    # else if the random error is 0, then it is an invalid request
    random_error = random.choice([1,1,1,1,0])
    
    # if the random error is 0, then manipulate the transaction id
    if random_error == 0:        
        if random.choice([0,1,0,0,0]) == 0:    
            transaction_id = transaction_id.replace('0', 's')        
        else:
            transaction_amount = transaction_amount + 100
    
    # return the values
    return ticket_id, user_name, transaction_id, transaction_amount, ticket_raise_time
    

---

#### `Connect to the database`

---

In [7]:
# connect to the database
db = mysql.connect(
    host = "localhost",
    user = "lakshay",  ## Enter your user name here
    password = "ABC@123", ## Enter your password here
    database = "website", ## Enter the database name here
    auth_plugin = "mysql_native_password",
)
# create the cursor
cursor = db.cursor()

Now, every 10-20 seconds a new random refund request will be generated. You can change this if you want. Also, after every 10 requests, that will take around 100-200 seconds, updated transaction id's list will selected.

---

In [ ]:
while True:
    # get the updated transaction_ids and price from the updated transactions table.
    command = "SELECT transaction_id, price FROM transaction"
    cursor.execute(command)
    transactions = cursor.fetchall()
    
    # generate 10 refund request every 10-20 seconds
    for i in range(10):
        command = "INSERT INTO refund_detail values" + str(generate_refund(transactions))
        cursor.execute(command)
        db.commit()
        time.sleep(random.randint(10,20))